**Inference к модели, захосченной через ngrok & LM Studio**

In [1]:
import requests
import json
from similarity import bm25_ensemble_with_crossenc_answer, prep_query

def message_llm(system_prompt, user_input=None, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            # { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
batch_1 = ""
batch_2 = ""
batch_3 = ""

rag_prompt = """
### CONTEXT ###
1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###
На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос. Если в приведенных данных контекст категории «ТАБЛИЦА»,
скажи о чем она, и предложи ознакомиться с ней, чтобы получить ответ на приведенный
ниже вопрос. Ты должен добавить в конце своего ответа ссылки на все используемые источники.

### USER ###
{QUESTION}"""

system_prompt = """
### ROLE ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы. Если из контекста 
невозможно дать ответ на вопрос, пиши «Я не могу ответить на этот вопрос»."""

# question = "На каком уровне ЦБ РФ поддерживает ставку инфляции? На что опирается ЦБ?"
# answer = message_llm(rag_prompt.replace('{QUESTION}', question))

/home/veidlink/HAKATONS/cb-purple-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [3]:
client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

In [7]:
question = "На каком уровне ЦБ РФ поддерживает ставку инфляции? На что опирается ЦБ?"
query = prep_query(question)
print(f'Prepped question: {query}')
bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=10, cr_enc_n_results=2, limit_knn=70, knn_docs_window=3)

Prepped question:  На каком уровне ЦБ РФ поддерживает ставку инфляции На что опирается ЦБ


IndexError: index out of range in self

In [35]:
print(answer)

Вопрос: На каком уровне ЦБ РФ поддерживает ставку инфляции?
